Hier läuft dann unser fertiges Retrieval System

In [ ]:
!pip3 install tira ir-datasets python-terrier

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

In [ ]:
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)
topics = pt_dataset.get_topics('text')
bm25 = pt.BatchRetrieve(index, wmodel="BM25", properties={"termpipelines" : "Stopwords,PorterStemmer"}) >> pt.pipelines.PerQueryMaxMinScoreTransformer()
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", properties={"termpipelines" : "Stopwords,PorterStemmer"}) >> pt.pipelines.PerQueryMaxMinScoreTransformer()

linear = 0.25 * bm25 + 0.75 * tfidf

run=linear(topics)
persist_and_normalize_run(run, system_name='retrieval_system', default_output='../runs')
print (run.head(10))

In [12]:
"cell for tests:"
bm25 = pt.BatchRetrieve(index, wmodel="BM25", properties={"termpipelines" : "Stopwords,PorterStemmer"}) >> pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = pt.BatchRetrieve(index, wmodel="DPH", properties={"termpipelines" : "Stopwords,PorterStemmer"}) >> pt.pipelines.PerQueryMaxMinScoreTransformer()
linear = 0.75 * bm25 + 0.25 * dph
run=linear(topics)
print (run.head(5))



  qid     docid                               docno     score  \
0   1   94858.0        2004.cikm_conference-2004.47  1.000000   
1   1  125137.0   1989.ipm_journal-ir0volumeA25A4.2  0.769343   
2   1  125817.0  2005.ipm_journal-ir0volumeA41A5.11  0.748299   
3   1    5868.0                            W05-0704  0.734252   
4   1   84876.0       2016.ntcir_conference-2016.90  0.718073   

                                      query  rank  
0  retrieval system improving effectiveness     0  
1  retrieval system improving effectiveness     1  
2  retrieval system improving effectiveness     2  
3  retrieval system improving effectiveness     3  
4  retrieval system improving effectiveness     4  
